In [1]:
"""
File v736_optima_elasticities.py

Michel Bierlaire
Mon Aug 31 17:38:50 2020

"""
import sys
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import Beta, Derive

# Read the data
df = pd.read_csv('optima.dat', '\t')
database = db.Database('optima', df)

# The following statement allows you to use the names of the
# variable as Python variable.
globals().update(database.variables)

# Define new variables
TimePT_scaled = TimePT / 200
TimeCar_scaled = TimeCar / 200
MarginalCostPT_scaled = MarginalCostPT / 10
CostCarCHF_scaled = CostCarCHF / 10
distance_km_scaled = distance_km / 5
male = Gender == 1
female = Gender == 2
unreportedGender = Gender == -1
fulltime = OccupStat == 1
notfulltime = OccupStat != 1

# Removing some observations
exclude = ((Choice == -1) + ((CarAvail == 3) * (Choice == 1))) != 0
database.remove(exclude)

# List of parameters to be estimated
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 0)
BETA_TIME_FULLTIME = Beta('BETA_TIME_FULLTIME', 0, None, None, 0)
BETA_TIME_OTHER = Beta('BETA_TIME_OTHER', 0, None, None, 0)
BETA_DIST_MALE = Beta('BETA_DIST_MALE', 0, None, None, 0)
BETA_DIST_FEMALE = Beta('BETA_DIST_FEMALE', 0, None, None, 0)
BETA_DIST_UNREPORTED = Beta('BETA_DIST_UNREPORTED', 0, None, None, 0)
BETA_COST = Beta('BETA_COST', 0, None, None, 0)

# Definition of utility functions:
V_PT = BETA_TIME_FULLTIME * TimePT_scaled * fulltime + \
    BETA_TIME_OTHER * TimePT_scaled * notfulltime + \
    BETA_COST * MarginalCostPT_scaled
V_CAR = ASC_CAR + \
    BETA_TIME_FULLTIME * TimeCar_scaled * fulltime + \
    BETA_TIME_OTHER * TimeCar_scaled * notfulltime + \
    BETA_COST * CostCarCHF_scaled
V_SM = ASC_SM + \
    BETA_DIST_MALE * distance_km_scaled * male + \
    BETA_DIST_FEMALE * distance_km_scaled * female + \
    BETA_DIST_UNREPORTED * distance_km_scaled * unreportedGender

# Associate utility functions with the numbering of alternatives
V = {0: V_PT,
     1: V_CAR,
     2: V_SM}

try:
    m = 'v715_optima_model'
    results = res.bioResults(pickleFile=f'{m}.pickle')
except FileNotFoundError:
    print(f'Run first the script {m}.py in order to generate the file '
          '{m}.pickle.')
    sys.exit()

prob_pt = models.logit(V, None, 0)
prob_car = models.logit(V, None, 1)
prob_sm = models.logit(V, None, 2)

# Statements for the calculation of the disaggregate direct elasticities
elas_pt_time = Derive(prob_pt, 'TimePT') * TimePT / prob_pt
elas_pt_cost = Derive(prob_pt, 'MarginalCostPT') * MarginalCostPT / prob_pt
elas_car_time = Derive(prob_car, 'TimeCar') * TimeCar / prob_car
elas_car_cost = Derive(prob_car, 'CostCarCHF') * CostCarCHF / prob_car
elas_sm_dist = Derive(prob_sm, 'distance_km') * distance_km / prob_sm

# Statements for the calculation of the disaggregate cross elasticities
elas_pt_time_car = Derive(prob_pt, 'TimeCar') * TimeCar / prob_pt
elas_pt_cost_car = Derive(prob_pt, 'CostCarCHF') * CostCarCHF / prob_pt
elas_car_time_pt = Derive(prob_car, 'TimePT') * TimePT / prob_car
elas_car_cost_pt = \
    Derive(prob_car, 'MarginalCostPT') * MarginalCostPT / prob_car

simulate = {'Prob. public transp.': prob_pt,
            'Prob. car': prob_car,
            'Prob. slow modes': prob_sm,
            'Disag. Elast. PT - Time': elas_pt_time,
            'Disag. Elast. PT - Cost': elas_pt_cost,
            'Disag. Elast. Car - Time': elas_car_time,
            'Disag. Elast. Car - Cost': elas_car_cost,
            'Disag. Elast. SM - Dist': elas_sm_dist,
            'Disag. Elast. PT - Time Car': elas_pt_time_car,
            'Disag. Elast. PT - Cost Car': elas_pt_cost_car,
            'Disag. Elast. Car - Time PT': elas_car_time_pt,
            'Disag. Elast. Car - Cost PT': elas_car_cost_pt,
            'Weight': Weight}

biosim = bio.BIOGEME(database, simulate)
biosim.modelName = 'v736_optima_elasticities'

# Perform the simulation
simresults = biosim.simulate(results.getBetaValues())

totalWeight = simresults['Weight'].sum()

simresults['Weighted public'] = simresults['Prob. public transp.'] * simresults['Weight']
simresults['Weighted car'] = simresults['Prob. car'] * simresults['Weight']
simresults['Weighted slow'] = simresults['Prob. slow modes'] * simresults['Weight']

# Denominators needed for the formula of aggregate elasticities
denominator_car = simresults['Weighted car'].sum()
denominator_pt = simresults['Weighted public'].sum()
denominator_sm = simresults['Weighted slow'].sum()

# Aggregate elasticities.
direct_elas_pt_time = (simresults['Weighted public']
                       * simresults['Disag. Elast. PT - Time']
                       / denominator_pt).sum()
print(f'Aggregate direct point elasticity of PT wrt time:\t'
      f'{direct_elas_pt_time:.4g}')

direct_elas_pt_cost = (simresults['Weighted public']
                       * simresults['Disag. Elast. PT - Cost']
                       / denominator_pt).sum()
print(f'Aggregate direct point elasticity of PT wrt cost:\t'
      f'{direct_elas_pt_cost:.4g}')

direct_elas_car_time = (simresults['Weighted car']
                        * simresults['Disag. Elast. Car - Time']
                        / denominator_car).sum()
print(f'Aggregate direct point elasticity of car wrt time:\t'
      f'{direct_elas_car_time:.4g}')

direct_elas_car_cost = (simresults['Weighted car']
                        * simresults['Disag. Elast. Car - Cost']
                        / denominator_car).sum()
print(f'Aggregate direct point elasticity of car wrt cost:\t'
      f'{direct_elas_car_cost:.4g}')

direct_elas_sm_dist = (simresults['Weighted slow']
                       * simresults['Disag. Elast. SM - Dist']
                       / denominator_sm).sum()
print(f'Aggregate direct point elasticity of SM wrt distance:\t'
      f'{direct_elas_sm_dist:.4g}')

cross_elas_pt_cartime = (simresults['Weighted public']
                         * simresults['Disag. Elast. PT - Time Car']
                         / denominator_pt).sum()
print(f'Aggregate cross point elasticity of PT wrt car time:\t'
      f'{cross_elas_pt_cartime:.4g}')

cross_elas_pt_carcost = (simresults['Weighted public']
                         * simresults['Disag. Elast. PT - Cost Car']
                         / denominator_pt).sum()
print(f'Aggregate cross point elasticity of PT wrt car cost:\t'
      f'{cross_elas_pt_carcost:.4g}')


cross_elas_car_pttime = (simresults['Weighted car']
                         * simresults['Disag. Elast. Car - Time PT']
                         / denominator_car).sum()
print(f'Aggregate cross point elasticity of car wrt PT time:\t'
      f'{cross_elas_car_pttime:.4g}')

cross_elas_car_ptcost = (simresults['Weighted car']
                         * simresults['Disag. Elast. Car - Cost PT']
                         / denominator_car).sum()
print(f'Aggregate cross point elasticity of car wrt PT cost:\t'
      f'{cross_elas_car_ptcost:.4g}')


Aggregate direct point elasticity of PT wrt time:	-0.2412
Aggregate direct point elasticity of PT wrt cost:	-0.3131
Aggregate direct point elasticity of car wrt time:	-0.04148
Aggregate direct point elasticity of car wrt cost:	-0.0948
Aggregate direct point elasticity of SM wrt distance:	-1.017
Aggregate cross point elasticity of PT wrt car time:	0.08852
Aggregate cross point elasticity of PT wrt car cost:	0.2056
Aggregate cross point elasticity of car wrt PT time:	0.1018
Aggregate cross point elasticity of car wrt PT cost:	0.1314


In [2]:
simresults['Disag. Elast. PT - Time']

0      -0.113034
2      -0.151123
3      -0.242360
4      -1.422994
5      -0.220470
          ...   
2259   -0.270663
2261   -0.969618
2262   -0.446238
2263   -0.233442
2264   -0.281503
Name: Disag. Elast. PT - Time, Length: 1899, dtype: float64